In [ ]:
# CNN Model
!pip install tensorflow tensorflow-gpu opencv-python matplotlib

In [ ]:
# list
!pip list

In [ ]:
# Importing tensor flow Library and os
import tensorflow as tf
import os

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
# Remove Dodgy Images
import cv2
import imghdr

In [ ]:
# data contains two folders in the simplest implementation. Should cover different data types for the photos.There can be more branches for data,
# a seperate folder for each category("Chicken","Pasta","ribs", etc)
#      (data)
#      /    \
#(Pizza)  (Not_pizza)
data_dir = 'data' 
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
# ----------------------------------------------------------------------
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [ ]:
# Loading Data
import numpy as np
from matplotlib import pyplot as plt

data = tf.keras.utils.image_dataset_from_directory('data')    # should  let you know how many files are read after this line

In [ ]:
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [ ]:
# analyzes the photos in a frame from 0-255 pixels 
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
#Scaling the data
data = data.map(lambda x,y: (x/255, y))
data.as_numpy_iterator().next()

In [ ]:
# Splitting the Data
rain_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
# Gives training size
train_size    

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
# Building Deep Learning Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
#---------------------------------------------------------------------------------
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='leaky_relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='leaky_relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='leaky_relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
# View Summary
model.summary()

In [ ]:
# Training the data
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(train, epochs=5, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
# Plotting Performance
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")

fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")

plt.show()

In [ ]:
# Evaluating Data
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
pre = Precision()
re = Recall()
acc = BinaryAccuracy()
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

In [ ]:
# Testing
import cv2

In [ ]:
img = cv2.imread('pizza.jpg')
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))   # convert BGR to RGB so we can see it normally 
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))    ## resizing image for testing 
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
y = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
# These might be switched, sry lol
if y > 0.5: 
    print(f'pizza')
else:
    print(f'not pizza')

In [ ]:
# Can Continue Testing From Here

# New Section